In [40]:
import pandas as pd
import numpy as np
import os
import timeit
from tqdm.notebook import trange, tqdm
import re
import matplotlib.pyplot as plt

In [41]:
DEMO = False
WRITE_ERRORS_TO_FILE = True

In [42]:

if DEMO:
    CHARTEVENTS_BY_ICUSTAY_ID = 'data_demo/parquet/'
    REDUCED_CE_BY_ICUSTAY_ID = 'data_demo/samples/'
    NUMERICAL_BY_ICUSTAY_ID = 'data_demo/samples_numerical/'
    STATB_BY_ICUSTAY_ID = 'data_demo/samples_statB/'
    STATB_CSV_BY_ICUSTAY_ID = 'data_demo/samples_statB_csv/'
else:
    CHARTEVENTS_BY_ICUSTAY_ID = 'data/parquet/'
    REDUCED_CE_BY_ICUSTAY_ID = 'data/samples/'
    NUMERICAL_BY_ICUSTAY_ID = 'data/samples_numerical/'
    STATB_BY_ICUSTAY_ID = 'data/samples_statB/'
    DEMO_BY_ICUSTAY_ID = 'data/samples_demographics/'


CHARTEVENTS_FILENAME = 'mimic-iii/CHARTEVENTS.csv'
READMISSION_FILENAME = 'data/readmission.csv'
PARQUET_EXT = '.parquet'
STEP_FAIL_FILE = 'Step7_failed.txt'

In [43]:
columns = ['ICUSTAY_ID', 'GENDER', 'AGE', 'ETHNICITY', 'INSURANCE']

In [44]:
readmission = pd.read_csv(READMISSION_FILENAME,usecols=columns).sort_values(['ICUSTAY_ID']).reset_index(drop=True)
print(readmission.dtypes)
readmission = readmission.set_index('ICUSTAY_ID')
readmission.head(1)

ICUSTAY_ID     int64
INSURANCE     object
ETHNICITY     object
GENDER        object
AGE            int64
dtype: object


,INSURANCE,ETHNICITY,GENDER,AGE
ICUSTAY_ID,,,,
200001,Medicare,ASIAN - ASIAN INDIAN,F,61


In [45]:
chartevents_dir_list = os.listdir(NUMERICAL_BY_ICUSTAY_ID)
len(chartevents_dir_list)

48075

In [46]:
failed = []
for stay in tqdm(chartevents_dir_list, total=len(chartevents_dir_list)):
    try:
        icustay_id = int(re.sub(r'[^0-9]*','',stay))
        demo_df = readmission.loc[[icustay_id]]
        #print(demo_df.head())
        demo_df.to_parquet(DEMO_BY_ICUSTAY_ID+stay)
        # break
    except Exception as e:
        failed.append(stay)
        failed.append(str(e))
        continue

  0%|          | 0/48075 [00:00<?, ?it/s]

In [47]:
failed

[]

In [48]:
if WRITE_ERRORS_TO_FILE:
    with open(STEP_FAIL_FILE, 'w') as f:
        for line in failed:
            f.write(line)
            f.write('\n')

In [49]:
demo_df.head()

,INSURANCE,ETHNICITY,GENDER,AGE
ICUSTAY_ID,,,,
254384,Medicare,WHITE,M,72
